In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [2]:
# Загрузка данных
data_class = load_breast_cancer()
X_class, y_class = data_class.data, data_class.target

# Разделение данных на тренировочную и тестовую выборки
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

# Бейзлайн модель решающего дерева
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train_class, y_train_class)
y_pred_class = tree_clf.predict(X_test_class)

# Оценка
accuracy_class = accuracy_score(y_test_class, y_pred_class)
precision_class = precision_score(y_test_class, y_pred_class)
recall_class = recall_score(y_test_class, y_pred_class)
f1_class = f1_score(y_test_class, y_pred_class)

print(f"Baseline Decision Tree Classification Metrics:\nAccuracy: {accuracy_class}\nPrecision: {precision_class}\nRecall: {recall_class}\nF1 Score: {f1_class}")

Baseline Decision Tree Classification Metrics:
Accuracy: 0.9473684210526315
Precision: 0.9577464788732394
Recall: 0.9577464788732394
F1 Score: 0.9577464788732394


In [8]:
from sklearn.datasets import fetch_california_housing
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [9]:
# Загрузка данных
data_reg = fetch_california_housing()
X_reg, y_reg = data_reg.data, data_reg.target

# Разделение данных на тренировочную и тестовую выборки
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# Бейзлайн модель решающего дерева для регрессии
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X_train_reg, y_train_reg)
y_pred_reg = tree_reg.predict(X_test_reg)

# Оценка
mae_reg = mean_absolute_error(y_test_reg, y_pred_reg)
mse_reg = mean_squared_error(y_test_reg, y_pred_reg)
r2_reg = r2_score(y_test_reg, y_pred_reg)

print(f"Baseline Decision Tree Regression Metrics:\nMAE: {mae_reg}\nMSE: {mse_reg}\nR-squared: {r2_reg}")

Baseline Decision Tree Regression Metrics:
MAE: 0.4557167466085271
MSE: 0.49969293809043114
R-squared: 0.6186740579561492


In [3]:
from sklearn.model_selection import GridSearchCV

# Параметры для перебора
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5, scoring='f1')
grid_search.fit(X_train_class, y_train_class)

# Лучший классификатор
best_tree_clf = grid_search.best_estimator_

# Оценка
y_pred_class_improved = best_tree_clf.predict(X_test_class)
accuracy_class_improved = accuracy_score(y_test_class, y_pred_class_improved)
precision_class_improved = precision_score(y_test_class, y_pred_class_improved)
recall_class_improved = recall_score(y_test_class, y_pred_class_improved)
f1_class_improved = f1_score(y_test_class, y_pred_class_improved)

print(f"Improved Decision Tree Classification Metrics:\nAccuracy: {accuracy_class_improved}\nPrecision: {precision_class_improved}\nRecall: {recall_class_improved}\nF1 Score: {f1_class_improved}")

Improved Decision Tree Classification Metrics:
Accuracy: 0.9473684210526315
Precision: 0.9577464788732394
Recall: 0.9577464788732394
F1 Score: 0.9577464788732394


In [10]:
# Параметры для перебора
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(DecisionTreeRegressor(random_state=42), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_reg, y_train_reg)

# Лучший регрессор
best_tree_reg = grid_search.best_estimator_

# Оценка
y_pred_reg_improved = best_tree_reg.predict(X_test_reg)
mae_reg_improved = mean_absolute_error(y_test_reg, y_pred_reg_improved)
mse_reg_improved = mean_squared_error(y_test_reg, y_pred_reg_improved)
r2_reg_improved = r2_score(y_test_reg, y_pred_reg_improved)

print(f"Improved Decision Tree Regression Metrics:\nMAE: {mae_reg_improved}\nMSE: {mse_reg_improved}\nR-squared: {r2_reg_improved}")

Improved Decision Tree Regression Metrics:
MAE: 0.4311152226926481
MSE: 0.40840458615620523
R-squared: 0.6883380738855668


In [7]:
import numpy as np


class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        # Если значение установлено, это узел-лист
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class CustomDecisionTreeClassifier:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.tree = None

    def fit(self, X, y):
        self.tree = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.tree) for x in X])

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # Остановки
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, n_features, replace=False)

        # Выбор лучшего сплита
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)

        if len(left_idxs) < self.min_samples_leaf or len(right_idxs) < self.min_samples_leaf:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        # Рост поддеревьев
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feat, best_thresh, left, right)

    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold
        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        # Вычисление прироста информации
        parent_entropy = self._entropy(y)
        left_idxs, right_idxs = self._split(X_column, split_thresh)
        n, n_left, n_right = len(y), len(left_idxs), len(right_idxs)
        if n_left == 0 or n_right == 0:
            return 0
        child_entropy = (n_left / n) * self._entropy(y[left_idxs]) + (n_right / n) * self._entropy(y[right_idxs])
        info_gain = parent_entropy - child_entropy
        return info_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log2(p) for p in ps if p > 0])

    def _most_common_label(self, y):
        counter = np.bincount(y)
        return np.argmax(counter)

    def _traverse_tree(self, x, node):
        if node.value is not None:
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

# Обучение и оценка кастомного дерева решений
data_class = load_breast_cancer()
X_class, y_class = data_class.data, data_class.target

X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

custom_tree_clf = CustomDecisionTreeClassifier(max_depth=10)
custom_tree_clf.fit(X_train_class, y_train_class)
y_pred_class_custom = custom_tree_clf.predict(X_test_class)

accuracy_class_custom = accuracy_score(y_test_class, y_pred_class_custom)
precision_class_custom = precision_score(y_test_class, y_pred_class_custom)
recall_class_custom = recall_score(y_test_class, y_pred_class_custom)
f1_class_custom = f1_score(y_test_class, y_pred_class_custom)

print(f"Custom Decision Tree Classification Metrics:\nAccuracy: {accuracy_class_custom}\nPrecision: {precision_class_custom}\nRecall: {recall_class_custom}\nF1 Score: {f1_class_custom}")

Custom Decision Tree Classification Metrics:
Accuracy: 0.9298245614035088
Precision: 0.9436619718309859
Recall: 0.9436619718309859
F1 Score: 0.9436619718309859


In [12]:
class CustomDecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.tree = None

    def fit(self, X, y):
        self.tree = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.tree) for x in X])

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        
        # Остановки
        if (depth >= self.max_depth or n_samples < self.min_samples_split):
            leaf_value = self._calculate_leaf_value(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, n_features, replace=False)

        # Выбор лучшего сплита
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)

        if len(left_idxs) < self.min_samples_leaf or len(right_idxs) < self.min_samples_leaf:
            leaf_value = self._calculate_leaf_value(y)
            return Node(value=leaf_value)

        # Рост поддеревьев
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feat, best_thresh, left, right)

    def _best_criteria(self, X, y, feat_idxs):
        best_mse = float('inf')
        split_idx, split_thresh = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)


            for threshold in thresholds:
                            mse = self._calculate_mse(y, X_column, threshold)
                            if mse < best_mse:
                                best_mse = mse
                                split_idx = feat_idx
                                split_thresh = threshold
        return split_idx, split_thresh

    def _calculate_mse(self, y, X_column, split_thresh):
        left_idxs, right_idxs = self._split(X_column, split_thresh)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return float('inf')
        y_left, y_right = y[left_idxs], y[right_idxs]
        mse = len(y_left) * np.var(y_left) + len(y_right) * np.var(y_right)
        return mse

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _calculate_leaf_value(self, y):
        return np.mean(y)

    def _traverse_tree(self, x, node):
        if node.value is not None:
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

# Обучение и оценка кастомного дерева решений для регрессии
custom_tree_reg = CustomDecisionTreeRegressor(max_depth=10)
custom_tree_reg.fit(X_train_reg, y_train_reg)
y_pred_reg_custom = custom_tree_reg.predict(X_test_reg)

mae_reg_custom = mean_absolute_error(y_test_reg, y_pred_reg_custom)
mse_reg_custom = mean_squared_error(y_test_reg, y_pred_reg_custom)
r2_reg_custom = r2_score(y_test_reg, y_pred_reg_custom)

print(f"Custom Decision Tree Regression Metrics:\nMAE: {mae_reg_custom}\nMSE: {mse_reg_custom}\nR-squared: {r2_reg_custom}")


Custom Decision Tree Regression Metrics:
MAE: 0.435403959596546
MSE: 0.4241351297691526
R-squared: 0.6763337730343438
